## Results Plotting Instructions:
1. Run all cells before the "CONFIG" cell
2. Edit the "CONFIG" cell with the desired parameters
3. Run the "CONFIG" cell and the "PLOT RESULTS" cell
4. Repeat steps 2 and 3 as desired

### See bottom of notebook for example parameters (to replicate Tim's results)

In [ ]:
import itertools
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import os
import seaborn as sns
import sqlite3

In [ ]:
cc = itertools.cycle(sns.color_palette('colorblind'))
color_map = {
    "simple_cnn": next(cc),
    "resnet": next(cc),
    "resnet_pretrained": next(cc),
    "resnet_pretrained_embeddings": next(cc),
    "resnet_no_pool": next(cc),
    "unseen": next(cc),
    "geo_mean_seen": next(cc),
    "geo_mean_unseen": next(cc)
}
model_name_map = {
    "simple_cnn": "2-Layer CNN",
    "resnet": "ResNet-18",
    "resnet_pretrained": "ResNet-18 (CIFAR-10 Pretrained)",
    "resnet_pretrained_embeddings": "ResNet-18 (CIFAR-10 Embeddings)",
    "resnet_no_pool": "ResNet-18 (Avg. Pooling Layer Removed)",
    "early_branching": "ResNet-18 (Early Branching)"
}

In [ ]:
def get_timestamps(input_name, model_name, output_name, experiment_name, num_results):
    conn = sqlite3.connect('../results/results_lookup.db')
    c = conn.cursor()
    timestamps = []
    for row in c.execute('''
        SELECT * FROM experiments 
            WHERE input=?
                AND output=?
                AND model=?
                AND experiment=?
                AND timestamp>'2020-01-21_19:00:00_000000'
    ''', (input_name, output_name, model_name, experiment_name)):
        timestamps.append(row[0])

    return timestamps[-1 * num_results:]

In [ ]:
def get_results(timestamps):
    train_results, test_results = [], []
    for trial_timestamp in timestamps:
        try:
            with open('../results/' + trial_timestamp + "/train.json") as train_fp:
                train_results.append(json.load(train_fp))
            with open('../results/' + trial_timestamp + "/test.json") as test_fp:
                test_results.append(json.load(test_fp))
        except:
            pass
#     for i in range(1, 11):
#         print(i)
#         for k, v in test_results[0][str(i)][-1].items():
#             print(k, v)
#         print("\n")
    return train_results, test_results
   
def get_keep_pcts(test_results):
    return sorted(list(test_results[0].keys()))[::-1]

def get_label_names(test_results, keep_pcts):
    return test_results[0][keep_pcts[0]][0]["class_1_name"].capitalize(), \
           test_results[0][keep_pcts[0]][0]["class_2_name"].capitalize()

In [ ]:
def get_left_out_accs(test_results, keep_pcts):
    left_out_final_num_accs_raw = np.array([[trial_test_results[keep_pct][-1]["left_out_num_acc"] for keep_pct in keep_pcts] for trial_test_results in test_results])
    left_out_final_num_accs = np.mean(left_out_final_num_accs_raw, axis=0)
    left_out_final_num_accs_std = np.std(left_out_final_num_accs_raw, axis=0)
    left_out_final_col_accs_raw = np.array([[trial_test_results[keep_pct][-1]["left_out_col_acc"] for keep_pct in keep_pcts] for trial_test_results in test_results])
    left_out_final_col_accs = np.mean(left_out_final_col_accs_raw, axis=0)
    left_out_final_col_accs_std = np.std(left_out_final_col_accs_raw, axis=0)
    return (left_out_final_num_accs, left_out_final_num_accs_std), \
           (left_out_final_col_accs, left_out_final_col_accs_std)

def get_non_left_out_accs(test_results, keep_pcts):
    non_left_out_final_num_accs_raw = np.array([[trial_test_results[keep_pct][-1]["non_left_out_num_acc"] for keep_pct in keep_pcts] for trial_test_results in test_results])
    non_left_out_final_num_accs = np.mean(non_left_out_final_num_accs_raw, axis=0)
    non_left_out_final_num_accs_std = np.std(non_left_out_final_num_accs_raw, axis=0)
    non_left_out_final_col_accs_raw = np.array([[trial_test_results[keep_pct][-1]["non_left_out_col_acc"] for keep_pct in keep_pcts] for trial_test_results in test_results])
    non_left_out_final_col_accs = np.mean(non_left_out_final_col_accs_raw, axis=0)
    non_left_out_final_col_accs_std = np.std(non_left_out_final_col_accs_raw, axis=0)
    return (non_left_out_final_num_accs, non_left_out_final_num_accs_std), \
           (non_left_out_final_col_accs, non_left_out_final_col_accs_std)

In [ ]:
def get_directory(input_name, model_name, output_name, experiment_name):
    directory = "../plots/{}/{}/{}/{}/".format(experiment_name, output_name, input_name, model_name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

def plot(keep_pcts, label_names, left_out_accs, non_left_out_accs, directory, model_name):   
    sns.set()
    sns.set_style('ticks')
    sns.set_context("poster")
    
    for i in range(len(label_names)):
        label_name = label_names[i]
        left_out_final_accs, left_out_final_accs_std = left_out_accs[i]
        non_left_out_final_accs, non_left_out_final_accs_std = non_left_out_accs[i]

        fig, ax = plt.subplots(figsize=(8, 6))
    
        ax.errorbar(keep_pcts[:0:-1], non_left_out_final_accs[:0:-1],
                    yerr=non_left_out_final_accs_std[:0:-1], zorder=10,
                    fmt='-s', clip_on=False, linewidth=4, ls='--', color=color_map[model_name])
        
        ax.errorbar(keep_pcts[:0:-1], left_out_final_accs[:0:-1], 
                    yerr=left_out_final_accs_std[:0:-1], zorder=10,
                    fmt='-o', clip_on=False, linewidth=4, color=color_map[model_name])
        
        ax.set_ylim(0, 1)
        ax.set_xlabel('Keep %')
        ax.set_ylabel('Test Accuracy')
        ax.set_title((r'Final Hold-Out $\bf {}$ Classification' + '\nAccuracy v. Keep %').format(label_name), fontsize=25, y=1.08)
#         ax.legend(["Seen Combinations", "Unseen Combinations"], loc='best', fontsize=20, framealpha=1,)

        # Set minor tick locations.
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.2))
        ax.xaxis.set_minor_locator(ticker.MultipleLocator(1))
        # Set grid to use minor tick locations. 
        ax.grid(which = 'both')
        plt.subplots_adjust(top=0.8, bottom=0.2, left=0.17)
        
        plt.xticks(keep_pcts[:0:-2], labels=[str(i + 1) + "/" + str(len(keep_pcts)) for i in range(0, len(keep_pcts), 2)])
        plt.yticks([x / 10. for x in range(0, 11, 2)])
        for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
            item.set_fontsize(20)
        fig.savefig(directory + "hold_out_{}_acc.pdf".format(label_name.lower()), dpi=500)
        plt.show()
        fig.clf()
        
def plot_geo_mean(keep_pcts, test_results, directory, model_name):
    sns.set()
    sns.set_style('ticks')
    sns.set_context("poster")

    left_out_final_num_accs_raw = np.array([[trial_test_results[keep_pct][-1]["left_out_num_acc"] for keep_pct in keep_pcts] for trial_test_results in test_results])
    left_out_final_num_accs = np.mean(left_out_final_num_accs_raw, axis=0)
    left_out_final_num_accs_std = np.std(left_out_final_num_accs_raw, axis=0)
    left_out_final_col_accs_raw = np.array([[trial_test_results[keep_pct][-1]["left_out_col_acc"] for keep_pct in keep_pcts] for trial_test_results in test_results])
    left_out_final_col_accs = np.mean(left_out_final_col_accs_raw, axis=0)
    left_out_final_col_accs_std = np.std(left_out_final_col_accs_raw, axis=0)

    non_left_out_final_num_accs_raw = np.array([[trial_test_results[keep_pct][-1]["non_left_out_num_acc"] for keep_pct in keep_pcts] for trial_test_results in test_results])
    non_left_out_final_num_accs = np.mean(non_left_out_final_num_accs_raw, axis=0)
    non_left_out_final_num_accs_std = np.std(non_left_out_final_num_accs_raw, axis=0)
    non_left_out_final_col_accs_raw = np.array([[trial_test_results[keep_pct][-1]["non_left_out_col_acc"] for keep_pct in keep_pcts] for trial_test_results in test_results])
    non_left_out_final_col_accs = np.mean(non_left_out_final_col_accs_raw, axis=0)
    non_left_out_final_col_accs_std = np.std(non_left_out_final_col_accs_raw, axis=0)
    
    left_out_geo_mean_acc_raw = np.sqrt(left_out_final_num_accs_raw * left_out_final_col_accs_raw)
    left_out_geo_mean_acc = np.mean(left_out_geo_mean_acc_raw, axis=0)
    left_out_geo_mean_acc_std = np.std(left_out_geo_mean_acc_raw, axis=0)
    
    non_left_out_geo_mean_acc_raw = np.sqrt(non_left_out_final_num_accs_raw * non_left_out_final_col_accs_raw)
    non_left_out_geo_mean_acc = np.mean(non_left_out_geo_mean_acc_raw, axis=0)
    non_left_out_geo_mean_acc_std = np.std(non_left_out_geo_mean_acc_raw, axis=0)

    fig, ax = plt.subplots(figsize=(8, 6))
    
    ax.errorbar(keep_pcts[:0:-1], non_left_out_geo_mean_acc[:0:-1],
                yerr=non_left_out_geo_mean_acc_std[:0:-1], zorder=10,
                fmt='-s', clip_on=False, linewidth=4, ls='--', color=color_map[model_name])
    
    ax.errorbar(keep_pcts[:0:-1], left_out_geo_mean_acc[:0:-1],
                yerr=left_out_geo_mean_acc_std[:0:-1], zorder=10,
                fmt='-o', clip_on=False, linewidth=4, color=color_map[model_name])
    
    ax.set_ylim(0, 1)
    ax.set_xlabel('Keep %')
    ax.set_ylabel('Test Accuracy')
    ax.set_title(r'$\bf{Geometric}$ $\bf{Mean}$ of' + '\nTest Accuracy v. Keep %', fontsize=25, y=1.08)
#     ax.legend(["Seen Combinations", "Unseen Combinations"], loc='best', fontsize=20, framealpha=1,)
    
    # Set minor tick locations.
    ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.2))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(1))
    # Set grid to use minor tick locations. 
    ax.grid(which = 'both')

#     print(keep_pcts[:0:-2])
    plt.xticks(keep_pcts[:0:-2], labels=[str(i + 1) + "/" + str(len(keep_pcts)) for i in range(0, len(keep_pcts), 2)])
#     plt.yticks([y / 4. for y in range(0, 5)])
    plt.yticks([x / 10. for x in range(0, 11, 2)])
    for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(20)
    plt.subplots_adjust(top=0.8, bottom=0.2, left=0.17)

    fig.savefig(directory + "geo_mean_acc.pdf", dpi=500)
    plt.show()
    fig.clf()

In [ ]:
def run_all(input_name, model_name, output_name, experiment_name, num_results):
    timestamps = get_timestamps(input_name, model_name, output_name, experiment_name, num_results)
    model_name = "resnet"
    _, test_results = get_results(timestamps)
    keep_pcts = get_keep_pcts(test_results)
    label_names = get_label_names(test_results, keep_pcts)
    left_out_accs = get_left_out_accs(test_results, keep_pcts)
    non_left_out_accs = get_non_left_out_accs(test_results, keep_pcts)

    directory = get_directory(input_name, model_name, output_name, experiment_name)
    plot(keep_pcts, label_names, left_out_accs, non_left_out_accs, directory, model_name)
    plot_geo_mean(keep_pcts, test_results, directory, model_name)

<br><br>
### CONFIG

In [ ]:
input_name =       # FILL IN
model_name =       # FILL IN
output_name =      # FILL IN
experiment_name =  # FILL IN

num_results = 10  # Number of experiment runs to average over for statistics

<br><br>
### PLOT RESULTS

In [ ]:
print("Task:", input_name, experiment_name)
print("Architecture:", model_name)
print("--input=" + input_name, "--output=" + output_name, "--experiment=" + experiment_name, "--model=" + model_name)
try:
    run_all(input_name, model_name, output_name, experiment_name)
except:
    print("FAILED")

<br><br>
### Additional code for plotting many experiments together in a single plot

In [ ]:
def setup_fig(n, ylabel, title):
#     plt.clf()
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.set_ylim(0, 1)
    ax.set_xlabel('Keep %')
    ax.set_ylabel(ylabel)
    ax.set_title(title, fontsize=25, y=1.08)

    # Set minor tick locations.
    ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.2))
#     ax.xaxis.set_minor_locator(ticker.MultipleLocator(1/9))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(1./n))
    # Set grid to use minor tick locations. 
    ax.grid(which = 'both')

    ax.set_yticks([x / 10. for x in range(0, 11, 2)])
    for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(20)

    return fig, ax

model_names = \
[
#    "simple_cnn",
   "resnet", 
#    "resnet_pretrained", 
#    "resnet_pretrained_embeddings", 
#    "resnet_no_pool"
]
                 
task_to_labels = {
    "left_out_colored_mnist": ("Shape", "Color"),
    "left_out_varied_location_mnist": ("Shape", "Position"),
    "left_out_scale_mnist": ("Shape", "Scale")
 }

keep_pct_map = {
    "left_out_colored_mnist": ['1', '0.9', '0.8', '0.7', '0.6', '0.5', '0.4', '0.3', '0.2', '0.1'],
    "left_out_varied_location_mnist": ['1.0', '0.8888888888888888', '0.7777777777777778', '0.6666666666666666', '0.5555555555555556', '0.4444444444444444', '0.3333333333333333', '0.2222222222222222', '0.1111111111111111']
    ,"left_out_scale_mnist": ['3', '2', '1']
}
model_to_name = {"simple_cnn" : "2-Layer CNN",
               "resnet": "ResNet-18", 
               "resnet_pretrained": "ResNet-18 (CIFAR10 Pretrained)", 
               "resnet_pretrained_embeddings": "ResNet-18 (CIFAR10 Embeddings)", 
               "resnet_no_pool": "ResNet-18 (No Avg. Pooling)"
    
}
def combined_plot():
      for input_name, experiment_name in [("left_out_colored_mnist", "keep_pct_readout_color_only"), 
                                    ("left_out_colored_mnist", "keep_pct_readout_shape_only"),
                                    ("left_out_varied_location_mnist", "keep_pct_readout_loc_only"),
                                   ("left_out_varied_location_mnist", "keep_pct_readout_shape_only"),
                                    ("left_out_scale_mnist", "keep_pct_readout_loc_only"),
                                         ("left_out_scale_mnist", "keep_pct_readout_shape_only")
                                         ]:  
        shape_fig, shape_ax = setup_fig(n, "Test Accuracy", (r'Final Hold-Out $\bf {}$ Classification' + '\nAccuracy v. Keep %').format(task_to_labels[input_name][0]))
        color_fig, color_ax = setup_fig(n, "Test Accuracy", (r'Final Hold-Out $\bf {}$ Classification' + '\nAccuracy v. Keep %').format(task_to_labels[input_name][1]))
        geo_m_fig, geo_m_ax = setup_fig(n, "Geometric Mean", r'$\bf{Geometric}$ $\bf{Mean}$ of' + '\nTest Accuracy v. Keep %')

        for ax in [shape_ax, color_ax, geo_m_ax]:
            ax.set_xticks(np.array(keep_pct_map[input_name][:0:-2], dtype=float), minor=False)
            ax.set_xticklabels(np.array([str(i + 1) + "/" + str(len(keep_pct_map[input_name])) for i in range(0, len(keep_pct_map[input_name]), 2)]), minor=False)
            
        for model_name in model_names:
            timestamps = get_timestamps(input_name, model_name, output_name, experiment_name)
            _, test_results = get_results(timestamps)
            keep_pcts = get_keep_pcts(test_results)
            directory = get_directory(input_name, model_name, output_name, experiment_name)
                 
            left_out_final_num_accs_raw = np.array([[trial_test_results[keep_pct][-1]["left_out_num_acc"] for keep_pct in keep_pcts] for trial_test_results in test_results])
            left_out_final_num_accs = np.mean(left_out_final_num_accs_raw, axis=0)
            left_out_final_num_accs_std = np.std(left_out_final_num_accs_raw, axis=0)
            left_out_final_col_accs_raw = np.array([[trial_test_results[keep_pct][-1]["left_out_col_acc"] for keep_pct in keep_pcts] for trial_test_results in test_results])
            left_out_final_col_accs = np.mean(left_out_final_col_accs_raw, axis=0)
            left_out_final_col_accs_std = np.std(left_out_final_col_accs_raw, axis=0)

            left_out_geo_mean_acc_raw = np.sqrt(left_out_final_num_accs_raw * left_out_final_col_accs_raw)
            left_out_geo_mean_acc = np.mean(left_out_geo_mean_acc_raw, axis=0)
            left_out_geo_mean_acc_std = np.std(left_out_geo_mean_acc_raw, axis=0)
            
            shape_ax.errorbar([float(x) for x in keep_pcts[:0:-1]], left_out_final_num_accs[:0:-1], 
                              yerr=left_out_final_num_accs_std[:0:-1], zorder=10,
                              fmt='-o', clip_on=False, linewidth=4, color=color_map[model_name],
                             label=model_to_name[model_name])
            color_ax.errorbar([float(x) for x in keep_pcts[:0:-1]], left_out_final_col_accs[:0:-1], 
                              yerr=left_out_final_col_accs_std[:0:-1], zorder=10,
                              fmt='-o', clip_on=False, linewidth=4, color=color_map[model_name],
                             label=model_to_name[model_name])
            geo_m_ax.errorbar([float(x) for x in keep_pcts[:0:-1]], left_out_geo_mean_acc[:0:-1], 
                              yerr=left_out_geo_mean_acc_std[:0:-1], zorder=10,
                              fmt='-o', clip_on=False, linewidth=4, color=color_map[model_name],
                             label=model_to_name[model_name])
                 
#         shape_ax.legend(fontsize=16)
        shape_fig.subplots_adjust(top=0.8, bottom=0.2, left=0.17)
        shape_fig.savefig("shape-" + input_name + ".pdf")
        shape_fig.savefig("shape-" + input_name + ".pdf")
        shape_fig.clf()
#         color_ax.legend(fontsize=16)
        color_fig.subplots_adjust(top=0.8, bottom=0.2, left=0.17)
        color_fig.savefig("color-" + input_name + ".pdf")
        color_fig.clf()
#         geo_m_ax.legend(fontsize=16, framealpha=1, loc='upper center', bbox_to_anchor=(0.5, -0.05),)
        geo_m_fig.subplots_adjust(top=0.8, bottom=0.2, left=0.17)
        geo_m_fig.savefig("geo_m-" + input_name + ".pdf")
        geo_m_fig.savefig("geo_m-" + input_name + ".png")
        geo_m_fig.clf()
        plt.show()
                    
combined_plot()

# Example parameters (to replicate Tim's thesis results)
### Chapter 4
* Shape-Color Problem
```python
input_name = "left_out_colored_mnist"
model_name = "resnet"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_default_v2"
```

* Shape-Location Problem
```python
input_name = "left_out_varied_location_mnist"
model_name = "resnet"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_9_class_v2"
```

* Shape-Scale Problem
```python
input_name = "left_out_many_scale_mnist"
model_name = "resnet"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_9_class_v2"
```

* Shape-Color-Location Problem
```python
input_name = "left_out_colored_location_mnist"
model_name = "resnet"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_3_task"
```

### Chapter 5 
Note: combined_plot() function above this cell can be used to display plots with multiple experiments)
##### Shape-Color Problem
* 2-Layer CNN
```python
input_name = "left_out_colored_mnist"
model_name = "simple_cnn"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_default_v2"
```

* ResNet-18
```python
input_name = "left_out_colored_mnist"
model_name = "resnet"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_default_v2"
```

* ResNet-18 (CIFAR10 Pretrained)
```python
input_name = "left_out_colored_mnist"
model_name = "resnet_pretrained"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_default_v2"
```

* ResNet-18 (CIFAR10 Embeddings)
```python
input_name = "left_out_colored_mnist"
model_name = "resnet_pretrained_embeddings"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_default_v2"
```

* ResNet-18 (No Avg. Pooling)
```python
input_name = "left_out_colored_mnist"
model_name = "resnet_no_pool"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_default_v2"
```

##### Shape-Location Problem
* 2-Layer CNN
```python
input_name = "left_out_varied_location_mnist"
model_name = "simple_cnn"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_9_class_v2"
```

* ResNet-18
```python
input_name = "left_out_varied_location_mnist"
model_name = "resnet"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_9_class_v2"
```

* ResNet-18 (CIFAR10 Pretrained)
```python
input_name = "left_out_varied_location_mnist"
model_name = "resnet_pretrained"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_9_class_v2"
```

* ResNet-18 (CIFAR10 Embeddings)
```python
input_name = "left_out_varied_location_mnist"
model_name = "resnet_pretrained_embeddings"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_9_class_v2"
```

* ResNet-18 (No Avg. Pooling)
```python
input_name = "left_out_varied_location_mnist"
model_name = "resnet_no_pool"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_9_class_v2"
```

### Chapter 6
##### Shape-Color Problem
* Shape Only "Branch"
```python
input_name = "left_out_colored_mnist"
model_name = "resnet"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_shape_only_default"
```

* Color Only "Branch"
```python
input_name = "left_out_colored_mnist"
model_name = "resnet"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_color_only"
```

##### Shape-Location Problem
* Shape
```python
input_name = "left_out_varied_location_mnist"
model_name = "resnet"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_shape_only_9_class"
```

* Location
```python
input_name = "left_out_varied_location_mnist"
model_name = "resnet"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_location_only"
```

##### Shape-Scale Problem
* Shape
```python
input_name = "left_out_many_scale_mnist"
model_name = "resnet"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_shape_only_9_class"
```

* Scale
```python
input_name = "left_out_many_scale_mnist"
model_name = "resnet"
output_name = "keep_pct_readout_dump"
experiment_name = "keep_pct_readout_location_only"  # Location-only logic works here
```